# Train gradient boosting model

# 1. Imports

## 1.1 Packages

In [1]:
import sys

import pandas as pd


## 1.2 Options

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../src')

from velib_prediction.pipelines.train_model.mlflow import (  # noqa: E402
    create_mlflow_experiment,
)
from velib_prediction.pipelines.train_model.nodes import (  # noqa: E402
    add_lags_sma,
    get_split_train_val_cv,
    train_model_cv_mlflow,
)


In [4]:
lags_to_try = [1,]

In [5]:
feat_date = "duedate"

## 1.3 Datasets

In [6]:
df_train = pd.read_parquet("../data/04_feature/df_feat_train.parquet")
df_train.sample(2)

,idx,stationcode,is_installed,capacity,numdocksavailable,numbikesavailable,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend
8,80501729403015,8050,1,45,41,3,1,2,1,1,2024-10-20 05:43:35+00:00,75056,2024,10,20,6,1
6,131011730672498,13101,1,34,27,6,1,5,1,1,2024-11-03 22:21:38+00:00,75056,2024,11,3,6,1


In [7]:
df_train.rename(columns={"numbikesavailable": "target"}, inplace=True)

# 2. Prepare datasets

In [8]:
# Add lags defined
df_train = add_lags_sma(df_train, lags_to_try, feat_id='stationcode', feat_date=feat_date, feat_target="target", n_shift=5)

In [9]:
df_train.sort_values(by="duedate", inplace=True)
df_train.drop(columns="duedate", inplace=True)

In [10]:
list_df = get_split_train_val_cv(df_train, n_splits=3)

In [11]:
len(list_df)

3

In [12]:
feat_cat = [
    "is_installed",
    "is_renting",
    "is_returning",
    "code_insee_commune",
    "duedate_weekend",
]

# 3. Train model

In [13]:
experiment_id = create_mlflow_experiment(
    experiment_folder_path="../data/06_models/mlruns",
    experiment_name="velib_prediction"
)
experiment_id

'587291553688351204'

In [14]:
params_catboost = {
    "iterations": 100,
    "depth": 7,
}

In [16]:
train_model_cv_mlflow(
    run_name="Test_catboost",
    experiment_id=experiment_id,
    list_train_valid=list_df,
    feat_cat=feat_cat,
    catboost_params=params_catboost,
    verbose=10,
)

/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/11/11 15:46:10 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: MlflowException("Failed to enforce schema of data '                 idx stationcode  is_i

Learning rate set to 0.204216
0:	learn: 8.2879177	test: 7.5947855	best: 7.5947855 (0)	total: 57.7ms	remaining: 5.71s
10:	learn: 1.9229949	test: 1.9464476	best: 1.9464476 (10)	total: 61.7ms	remaining: 499ms
20:	learn: 0.7498882	test: 1.2583431	best: 1.2583431 (20)	total: 65.7ms	remaining: 247ms
30:	learn: 0.5504552	test: 1.1492577	best: 1.1492577 (30)	total: 70.2ms	remaining: 156ms
40:	learn: 0.4386838	test: 1.0906112	best: 1.0906112 (40)	total: 73.9ms	remaining: 106ms
50:	learn: 0.3628740	test: 1.0448733	best: 1.0448733 (50)	total: 78.4ms	remaining: 75.3ms
60:	learn: 0.3082720	test: 1.0204172	best: 1.0204172 (60)	total: 82ms	remaining: 52.4ms
70:	learn: 0.2599096	test: 1.0031083	best: 1.0030558 (69)	total: 85.9ms	remaining: 35.1ms
80:	learn: 0.2211429	test: 0.9847377	best: 0.9847377 (80)	total: 89.9ms	remaining: 21.1ms
90:	learn: 0.2015777	test: 0.9738408	best: 0.9738408 (90)	total: 93.4ms	remaining: 9.23ms
99:	learn: 0.1803074	test: 0.9675743	best: 0.9675743 (99)	total: 97.5ms	remaini

/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/11/11 15:46:12 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: MlflowException("Failed to enforce schema of data '                  idx stationcode  is_

Learning rate set to 0.227694
0:	learn: 7.8645281	test: 8.1732645	best: 8.1732645 (0)	total: 9.09ms	remaining: 900ms
10:	learn: 1.4827616	test: 2.9689809	best: 2.9689809 (10)	total: 15.7ms	remaining: 127ms
20:	learn: 0.7437128	test: 2.2049915	best: 2.2049915 (20)	total: 21.6ms	remaining: 81.1ms
30:	learn: 0.5685405	test: 2.0965980	best: 2.0965980 (30)	total: 27.2ms	remaining: 60.4ms
40:	learn: 0.4510357	test: 2.0338918	best: 2.0338918 (40)	total: 32ms	remaining: 46.1ms
50:	learn: 0.3764778	test: 1.9528864	best: 1.9519673 (49)	total: 37.1ms	remaining: 35.7ms
60:	learn: 0.3139373	test: 1.9338885	best: 1.9243468 (58)	total: 42.3ms	remaining: 27ms
70:	learn: 0.2666599	test: 1.9357011	best: 1.9243468 (58)	total: 49.4ms	remaining: 20.2ms
80:	learn: 0.2340987	test: 1.9181725	best: 1.9139059 (78)	total: 55.1ms	remaining: 12.9ms
90:	learn: 0.2071453	test: 1.8987260	best: 1.8987260 (90)	total: 60.2ms	remaining: 5.95ms
99:	learn: 0.1880245	test: 1.8957842	best: 1.8942637 (97)	total: 66ms	remainin

/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/11/11 15:46:13 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: MlflowException("Failed to enforce schema of data '                  idx stationcode  is_

Learning rate set to 0.24266
0:	learn: 7.6173193	test: 7.5281582	best: 7.5281582 (0)	total: 798us	remaining: 79.1ms
10:	learn: 1.3328502	test: 2.3253323	best: 2.3253323 (10)	total: 7.2ms	remaining: 58.2ms
20:	learn: 0.7378684	test: 1.7040665	best: 1.7040665 (20)	total: 13.7ms	remaining: 51.4ms
30:	learn: 0.6228388	test: 1.5180303	best: 1.5180303 (30)	total: 20.8ms	remaining: 46.4ms
40:	learn: 0.5400639	test: 1.3907635	best: 1.3907635 (40)	total: 26.8ms	remaining: 38.5ms
50:	learn: 0.4704229	test: 1.3065026	best: 1.3065026 (50)	total: 69ms	remaining: 66.3ms
60:	learn: 0.3872695	test: 1.2378014	best: 1.2378014 (60)	total: 75.8ms	remaining: 48.5ms
70:	learn: 0.3272282	test: 1.1814235	best: 1.1814235 (70)	total: 82.7ms	remaining: 33.8ms
80:	learn: 0.2875514	test: 1.1551786	best: 1.1551786 (80)	total: 88.9ms	remaining: 20.9ms
90:	learn: 0.2532266	test: 1.1375581	best: 1.1375581 (90)	total: 94.9ms	remaining: 9.39ms
99:	learn: 0.2278953	test: 1.1260603	best: 1.1260603 (99)	total: 100ms	remain